# Introduction to pyjeo

**Material provided by Dr. Pieter Kempeneers (European Commission, Joint Research Centre)**

[pyjeo](https://pyjeo.readthedocs.io) is the follow up of [PKTOOLS](http://pktools.nongnu.org/html/index.html), a suite of utilities written in C++ for image processing with a focus on remote sensing applications. It is distributed under a General Public License (GPLv3) as a Python package.

In a nutshell, the main differences between pyjeo and pktools from a user's perspective are:

- pyjeo is a Python package should be run in a **Python environment**, whereas pktools applications are run from the command line (e.g., in a bash shell)
- pyjeo runs with images entirely **in memory**, whereas pktools runs most applications line per line. This makes pyjeo considerably faster, but with a larger memory footprint. However, there are some methods implemented in pyjeo to reduce the memory footprint by tiling the image


Run the following script to perform the installation of pyjeo

    sudo apt update
    sudo apt upgrade -y  
    cd ~/Downloads
    wget https://raw.githubusercontent.com/selvaje/SE_data/main/exercise/install_pyjeo.sh
    sudo install_pyjeo.sh



Import Python modules

In [3]:
# Trick to consider for venv within a notebook: simply use a clean sys.path taken from the shell
import sys
sys.path

['/media/sf_my_SE_data/exercise',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/user/.local/lib/python3.10/site-packages',
 '/home/user/.local/lib/python3.10/site-packages/pyjeo-1.1.2-py3.10.egg',
 '/home/user/.local/lib/python3.10/site-packages/rioxarray-0.15.5-py3.10.egg',
 '/home/user/.local/lib/python3.10/site-packages/pyproj-3.6.1-py3.10-linux-x86_64.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages/IPython/extensions',
 '/home/user/.ipython']

In [5]:
sys.path = ['', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/home/user/venv/lib/python3.10/site-packages', '/home/user/.local/lib/python3.10/site-packages', '/home/user/.local/lib/python3.10/site-packages/pyjeo-1.1.2-py3.10.egg', '/home/user/.local/lib/python3.10/site-packages/rioxarray-0.15.5-py3.10.egg', '/home/user/.local/lib/python3.10/site-packages/pyproj-3.6.1-py3.10-linux-x86_64.egg', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3.10/dist-packages']

In [6]:
sys.path

['',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '/home/user/venv/lib/python3.10/site-packages',
 '/home/user/.local/lib/python3.10/site-packages',
 '/home/user/.local/lib/python3.10/site-packages/pyjeo-1.1.2-py3.10.egg',
 '/home/user/.local/lib/python3.10/site-packages/rioxarray-0.15.5-py3.10.egg',
 '/home/user/.local/lib/python3.10/site-packages/pyproj-3.6.1-py3.10-linux-x86_64.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3.10/dist-packages']

In [15]:
# from now on all should work as expected...

In [11]:
from pathlib import Path
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import pyjeo as pj

Call the inline help function for [pj.geometry.warp](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry.warp)

In [12]:
help(pj.geometry.warp)

Help on function warp in module pyjeo.modules.geometry:

warp(jim_object, t_srs=None, bbox: list = None, ulx: float = None, uly: float = None, ulz: float = None, lrx: float = None, lry: float = None, lrz: float = None, dx: float = None, dy: float = None, **kwargs)
    Warp a raster dataset to a target spatial reference system.
    
    :param jim_object: a Jim object
    :param t_srs: Target spatial reference system
    :param bbox: bounding box (instead of ulx, uly, lrx, lry)
    :param ulx: Upper left x value of bounding box
    :param uly: Upper left y value of bounding box
    :param lrx: Lower right x value of bounding box
    :param lry: Lower right y value of bounding box
    :param dx: spatial resolution in x)
    :param dy: spatial resolution in y)
    :param kwargs: See table below
    :return: warped image as Jim instance
    
    
    +----------+---------------------------------------------------------------------------------------------------+
    | key      | value      

## Jim: a geospatial raster dataset object

Open an image and show its properties

In [29]:
jim = pj.Jim('geodata/vegetation/ETmean08-11.tif')

In [30]:
jim.properties.nrOfCol()

720

A Jim object is a geospatial dataset within a coordinate reference system

In [31]:
jim.properties.getProjection()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

From [gdal](https://gdal.org/tutorials/geotransforms_tut.html#introduction-to-geotransforms): "A geotransform is an affine transformation from the image coordinate space (row, column), also known as (pixel, line) to the georeferenced coordinate space (projected or geographic coordinates)"

Geotransform is an array with 6 items:

[0] top left x

[1] w-e pixel resolution

[2] rotation, 0 if image is “north up”

[3] top left y

[4] rotation, 0 if image is “north up”

[5] n-s pixel resolution

In [32]:
jim.properties.getGeoTransform()

[29.0, 0.00833333334, 0.0, 4.000000004, 0.0, -0.00833333334]

We have a single band and single plane raster object

In [33]:
jim.properties.nrOfBand()

1

In [34]:
jim.properties.nrOfPlane()

1

Open a second GeoTIFF image of the same dimension

In [35]:
gpp = pj.Jim('geodata/vegetation/GPPmean08-11.tif')

Stack this image to the existing bands of jim, creating a multi-band jim object

In [36]:
jim.geometry.stackBand(gpp)

In [37]:
jim.properties.nrOfBand()

2

## Exercise 1: stackBand function vs. method

Create a new jim object `jim_stacked` using the [stackBand](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry.stackBand) function instead of the [stackBand](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry._Geometry.stackBand) method. It should contain two bands, one for `ETmean08-11.tif` and one for `GPPmean08-11.tif`.

In [38]:
jim_stacked = pj.geometry.stackBand(jim, gpp)

In [39]:
jim_stacked.properties.nrOfBand()

3

## Labeled dimensions

Let's label the dimensions

In [17]:
bandnames = ['ET', 'GPP']
planenames = [datetime.strptime('2019-08-11','%Y-%m-%d')]
jim.properties.setDimension({'band' : bandnames, 'plane' : planenames})

In [18]:
jim.properties.getDimension('band')

['ET', 'GPP']

In [20]:
len(jim.properties.getDimension('plane'))

1

## Exercise 2: cropBand using numbered and labeled index

Crop second band as a new image (index starts from 0)

In [21]:
jim1 = pj.geometry.cropBand(jim, 1)

Create a new jim object `gpp` that contains the second band using the [cropBand]() function and the labeled index ('GPP')

In [22]:
gpp = pj.geometry.cropBand(jim, 'GPP')

Check if results are identical

In [23]:
jim1.properties.isEqual(gpp)

True

In [24]:
jim1.properties.getDimension()

{'plane': [datetime.datetime(2019, 8, 11, 0, 0)], 'band': ['GPP']}

## Working with multi-plane Jim objects

First convert the bands to planes using [band2plane](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry._Geometry.band2plane)

In [25]:
jim.geometry.band2plane()

The labels of band are copied to plane

In [26]:
jim.properties.getDimension()

{'band': [], 'plane': ['ET', 'GPP']}

## Calculate composites via reducePlane

Reduce the planes by calculating the mean of all planes

In [23]:
jim_mean = pj.geometry.reducePlane(jim, 'mean')

Where is the RuntimeWarning coming from?

In [24]:
jim.stats.getStats()

{'min': 0.0, 'max': 7.492255210876465, 'mean': 2.520120143890381}

In [25]:
jim[jim < 0] = 0
jim.stats.getStats()

{'min': 0.0, 'max': 7.492255210876465, 'mean': 2.520120143890381}

In [26]:
jim_mean = pj.geometry.reducePlane(jim, 'mean')

### Calculate custom composites via callback function in reducePlane

In [27]:
def getMax(reduced, plane):
    return pj.pixops.supremum(reduced, plane)
jim_max = pj.geometry.reducePlane(jim, getMax)

Crop the plane `ET` from `jim` using the method [cropPlane](https://pyjeo.readthedocs.io/en/latest/3_reference.html#geometry._Geometry.cropPlane). Check the resulting `jim` is a single plane and single band Jim object

In [28]:
jim.geometry.cropPlane('ET')
print(jim.properties.nrOfPlane())
print(jim.properties.nrOfBand())

JimPlanesError: plane not supported, set plane dimension or use integer

In [ ]:
jim.geometry.plane2band()
bandnames = ['ET']
planenames = [datetime.strptime('2019-08-11','%Y-%m-%d')]
jim.properties.setDimension({'band' : bandnames, 'plane' : planenames})

## Bridging to third party libraries: Numpy

In [40]:
jim.np()

array([[ 3.9031930e+00,  4.1704483e+00,  4.4033966e+00, ...,
        -3.4000000e+38,  8.0441576e-01,  7.9959238e-01],
       [ 3.9995923e+00,  3.7383151e+00,  4.3366170e+00, ...,
         7.8573370e-01,  8.0271739e-01,  8.0427992e-01],
       [ 3.8954484e+00,  4.0010872e+00,  4.0186820e+00, ...,
         7.9300272e-01,  8.0869567e-01,  8.2391304e-01],
       ...,
       [ 3.9974184e+00,  3.9528532e+00,  2.3230300e+00, ...,
         3.3593750e+00,  3.2286685e+00,  3.0525136e+00],
       [ 3.8603940e+00,  2.3833561e+00,  2.3200407e+00, ...,
         3.5756793e+00,  3.2909646e+00,  3.3271060e+00],
       [ 3.8717391e+00,  3.2824049e+00,  3.4489131e+00, ...,
         3.6081522e+00,  3.5593750e+00,  3.5692935e+00]], dtype=float32)

Check the type of jim.np()

In [41]:
type(jim.np())

numpy.ndarray

Use [sum](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.sum.html#numpy-ndarray-sum) function of Numpy to assert there are no pixels where `jim_max < jim_mean`

In [42]:
(jim_max < jim_mean).np().sum()

0

### Get items

With pyjeo we create the masks in memory in a "pythonic" way using [get items](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#get-jim-items) without the need to write temporary files.

Get sub-dataset based on pixel coordinates (first 3 rows and columns, starting from 0)

In [43]:
jim[0:3,0:3]

Last 3 rows and columns, show geographic bounding box [ulx, uly, lrx, lry]

In [44]:
jim[-3:,-3:].properties.getBBox()

[34.97500000478, -0.9749999999800003, 35.0000000048, -1.0000000000000002]

In [45]:
jim[0:3,0:3].np()

array([[3.903193 , 4.1704483, 4.4033966],
       [3.9995923, 3.738315 , 4.336617 ],
       [3.8954484, 4.001087 , 4.018682 ]], dtype=float32)

### Set items

In pyjeo, we can apply the mask in a "pythonic" way using [set items](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#set-jim-items)

In [46]:
jim[0:3,0:3] = 0
jim[0:5,0:5].np()

array([[0.       , 0.       , 0.       , 4.5625   , 4.419769 ],
       [0.       , 0.       , 0.       , 3.9793477, 4.114334 ],
       [0.       , 0.       , 0.       , 4.02894  , 3.8538043],
       [4.3265624, 4.3577447, 4.1216035, 3.8828125, 3.6519022],
       [3.9654212, 4.1422553, 4.012704 , 4.0539403, 3.761413 ]],
      dtype=float32)

**Display image using matplotlib**

We can show an image with matplotlib by providing a Numpy representation of the dataset

In [ ]:
# plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
ax1.imshow(jim.np())
plt.show()

## Calculate median filter via scipy

In [ ]:
from scipy import ndimage
npimage = ndimage.median_filter(jim.np(), size = 5)

In [ ]:
# plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
ax1.imshow(npimage)
plt.show()

## Exercise 4: Create a geospatial Jim object with CRS from a Numpy array

**A numpy array is not a geospatial dataset with a spatial coordinate reference system !**

In [ ]:
ajim = pj.np2jim(npimage)
print(ajim.properties.getProjection())
print(ajim.properties.getGeoTransform())

Use [setProjection](https://pyjeo.readthedocs.io/en/latest/3_reference.html#properties._Properties.setProjection) and [getProjection](https://pyjeo.readthedocs.io/en/latest/3_reference.html#properties._Properties.getProjection) to set the Coordinate Reference System (CRS) of ajim

## Calculating the median_filter in place using the output parameter

In [ ]:
ndimage.median_filter(jim.np(), output = jim.np(), size = 5)

As an alternative, we can set the Numpy representation of a Jim object in place

In [ ]:
jim.np()[:] = ndimage.median_filter(jim.np(), size = 5)

## Bridging to third party libraries: Xarray

In [ ]:
jim.xr()

**An Xarray dataset is defined with a spatial coordinate reference system !**

In [ ]:
ajim = pj.xr2jim(jim.xr())

In [ ]:
print(ajim.properties.getProjection())
print(ajim.properties.getGeoTransform())

At the basis we find a Numpy data array

In [ ]:
type(jim.xr().ET.data)

### Calculate median filter via Xarray

Processing can take several minutes...

In [ ]:
# xr_median = jim.xr().ET.rolling(x = 3, y = 3, center=True).median()

In [ ]:
# xr_median.plot()

**Notice that XArray member functions return the processed result and do not alter the input image !**

## Bridging JimVect to third party libraries

In [ ]:
sample = pj.JimVect('geodata/shp/polygons.sqlite')

In [ ]:
import pandas as pd
pdf = pd.DataFrame(sample.dict())
pdf

## Data extraction and regional statistics

In [ ]:
extracted = pj.geometry.extract(sample, jim, rule='mean', output='/vsimem/mean.json', oformat='GeoJSON')

In [ ]:
pdf = pd.DataFrame(extracted.dict())
pdf